## Junção de dados

In [97]:
import numpy as np
from scipy.spatial import distance
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
usecols = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'PERIDOOCORRENCIA', 
           'LOGRADOURO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 
           'LONGITUDE', 'DELEGACIA_CIRCUNSCRICAO']
drop_list = ['ANO_BO', 'NUM_BO', 'DELEGACIA_CIRCUNSCRICAO']

In [3]:
# NOTE: PERIODOOCORRENCIA pode ser desconhecido = "EM HORA INCERTA"
fc = pd.read_csv('data/FURTO DE CELULAR.csv', usecols=usecols)
fc = fc.drop_duplicates(drop_list)

/home/igor.lima/anaconda3/envs/dl_coursera/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
rc = pd.read_csv('data/ROUBO DE CELULAR.csv', usecols=usecols)
rc = rc.drop_duplicates(drop_list)

In [5]:
fv = pd.read_csv('data/FURTO DE VEÍCULOS.csv', usecols=usecols)
fv = fv.drop_duplicates(drop_list)

In [6]:
rv = pd.read_csv('data/ROUBO DE VEÍCULOS.csv', usecols=usecols,  )
rv = rv.drop_duplicates(drop_list)

In [104]:
def fix_latlong(x):
    try:
        return float(x)
    except:
        return float(x.replace(',', '.'))

In [105]:
df = pd.concat([fc, rc, fv,  rv], ignore_index=True).dropna()
df['BO_INICIADO'] = pd.to_datetime(df['BO_INICIADO'], errors='coerce')
df['LOGRADOURO'] = df['LOGRADOURO'].apply(lambda x: x.upper())
df['BAIRRO'] = df['BAIRRO'].apply(lambda x: x.upper())
df['DELEGACIA_CIRCUNSCRICAO'] = df['DELEGACIA_CIRCUNSCRICAO'].apply(lambda x: x.upper())
df['LATITUDE'] = df['LATITUDE'].apply(fix_latlong)
df['LONGITUDE'] = df['LONGITUDE'].apply(fix_latlong)

# drop NaT
df = df.dropna()
df.head()

,ANO_BO,NUM_BO,BO_INICIADO,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DELEGACIA_CIRCUNSCRICAO
0,2017,1,2017-01-01 00:28:58,DE MADRUGADA,AVENIDA PAULISTA,CONSOLACAO,S.PAULO,SP,-23.559153,-46.658927,78º D.P. JARDINS
1,2017,4,2017-01-01 02:05:37,DE MADRUGADA,RUA DOUTOR VITAL BRASIL,PROL JARDIM PAULISTA,FRANCA,SP,-20.522797,-47.372208,03º D.P. FRANCA
2,2017,1,2017-01-01 03:24:38,EM HORA INCERTA,AVENIDA CÁSPER LÍBERO,PQ RIBEIRÃO PRETO,RIBEIRAO PRETO,SP,-21.203052,-47.849607,06º D.P. RIBEIRAO PRETO
3,2017,4,2017-01-01 03:22:21,A NOITE,AVENIDA RIO BRANCO,REPUBLICA,S.PAULO,SP,-23.538027,-46.641257,03º D.P. CAMPOS ELISEOS
4,2017,2,2017-01-01 03:28:30,EM HORA INCERTA,AVENIDA MARANHÃO,CENTRO,CARAGUATATUBA,SP,-23.627634,-45.417505,DEL.POL.CARAGUATATUBA


In [106]:
df_count = df[df['ANO_BO'] == 2018].groupby([df.BO_INICIADO, df.LOGRADOURO]).size().reset_index(name="count")
df_count.head()

,BO_INICIADO,LOGRADOURO,count
0,2018-01-01 00:35:24,RUA AFFONSO RAIOLA,1
1,2018-01-01 00:48:34,RUA FREDERICO KRAHENBUHL,1
2,2018-01-01 01:02:32,RUA LUIZ ANTÔNIO DE PAIVA,1
3,2018-01-01 01:04:10,AVENIDA ACRE,1
4,2018-01-01 01:04:10,RUA CARIJÓS,1


In [107]:
export = df[df.ANO_BO == 2018].merge(df_count, how='left').sort_values('BO_INICIADO').reset_index(drop=True)
export.head()

,ANO_BO,NUM_BO,BO_INICIADO,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DELEGACIA_CIRCUNSCRICAO,count
0,2018,1,2018-01-01 00:35:24,A NOITE,RUA AFFONSO RAIOLA,RIO PEQUENO,S.PAULO,SP,-23.573792,-46.780034,75º D.P. JARDIM ARPOADOR,1
1,2018,7,2018-01-01 00:48:34,DE MADRUGADA,RUA FREDERICO KRAHENBUHL,CASTELINHO,PIRACICABA,SP,-22.730370,-47.671252,07º D.P. PIRACICABA,1
2,2018,11,2018-01-01 01:02:32,A NOITE,RUA LUIZ ANTÔNIO DE PAIVA,VILA JAMIL,FERRAZ DE VASCONCELOS,SP,-23.543338,-46.369557,DEL.POL.FERRAZ DE VASCONCELOS,1
3,2018,1,2018-01-01 01:04:10,A NOITE,RUA CARIJÓS,VILA LAZIRA,S.ANDRE,SP,-23.671147,-46.519664,03º D.P. SANTO ANDRÉ,1
4,2018,1,2018-01-01 01:04:10,A NOITE,AVENIDA ACRE,SAO JUDAS TADEU,BARRETOS,SP,-20.538186,-48.564419,03º D.P. BARRETOS,1


In [108]:
export.to_csv('clean.csv', index=False)